In [2]:
data = [
    ("I am very happy today", "positive"),
    ("This is an excellent product", "positive"),
    ("I feel amazing", "positive"),
    ("I love this service", "positive"),

    ("I am very sad", "negative"),
    ("This is the worst experience", "negative"),
    ("I feel angry", "negative"),
    ("I hate this app", "negative"),

    ("It is okay", "neutral"),
    ("Nothing special", "neutral"),
    ("Average experience", "neutral"),
    ("I am not sure", "neutral")
]

texts = [x[0] for x in data]
labels = [x[1] for x in data]


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(texts)

lr_model = LogisticRegression()
lr_model.fit(X, labels)

def predict_lr(text):
    return lr_model.predict(vectorizer.transform([text]))[0]


In [4]:
from sklearn.svm import LinearSVC

svm_model = LinearSVC()
svm_model.fit(X, labels)

def predict_svm(text):
    return svm_model.predict(vectorizer.transform([text]))[0]


In [5]:
pip install tensorflow


In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.preprocessing import LabelEncoder

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
X_dl = pad_sequences(sequences, maxlen=20)

le = LabelEncoder()
y_dl = le.fit_transform(labels)

model_lstm = Sequential([
    Embedding(5000, 64, input_length=20),
    LSTM(64),
    Dense(3, activation='softmax')
])

model_lstm.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model_lstm.fit(X_dl, y_dl, epochs=10, verbose=0)

def predict_lstm(text):
    seq = tokenizer.texts_to_sequences([text])
    pad = pad_sequences(seq, maxlen=20)
    pred = model_lstm.predict(pad)
    return le.inverse_transform([pred.argmax()])[0]


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [7]:
pip install transformers torch


In [8]:
from transformers import pipeline

bert_sentiment = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english"
)

def predict_bert(text):
    result = bert_sentiment(text)[0]
    if result['label'] == "POSITIVE":
        return "positive"
    else:
        return "negative"


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


In [9]:
import random

responses = {
    "positive": [
        "😊 That's wonderful to hear!",
        "👍 I'm glad you're feeling good!",
        "😄 Awesome! How can I help more?"
    ],
    "negative": [
        "😟 I'm sorry you're feeling this way.",
        "💬 I understand your frustration.",
        "🤝 I'm here to help you."
    ],
    "neutral": [
        "🙂 Okay, I understand.",
        "📌 Got it.",
        "🤖 Tell me more."
    ]
}

def respond(sentiment):
    return random.choice(responses[sentiment])


In [10]:
def chatbot(model_type="lr"):
    print("🤖 Sentiment-Aware Chatbot")
    print("Models: lr | svm | lstm | bert")
    print("Type 'exit' to quit\n")

    while True:
        text = input("You: ")
        if text.lower() == "exit":
            print("Bot: 👋 Goodbye!")
            break

        if model_type == "lr":
            sentiment = predict_lr(text)
        elif model_type == "svm":
            sentiment = predict_svm(text)
        elif model_type == "lstm":
            sentiment = predict_lstm(text)
        elif model_type == "bert":
            sentiment = predict_bert(text)
        else:
            sentiment = "neutral"

        print("Detected Sentiment:", sentiment)
        print("Bot:", respond(sentiment))
        print("-" * 50)


# Run chatbot
chatbot(model_type="lr")   # change to svm / lstm / bert


🤖 Sentiment-Aware Chatbot
Models: lr | svm | lstm | bert
Type 'exit' to quit

You: Hi
Detected Sentiment: positive
Bot: 😄 Awesome! How can I help more?
--------------------------------------------------
You: bad
Detected Sentiment: positive
Bot: 😊 That's wonderful to hear!
--------------------------------------------------
You: sorry
Detected Sentiment: positive
Bot: 😊 That's wonderful to hear!
--------------------------------------------------
You:  I'm here to help you.
Detected Sentiment: positive
Bot: 👍 I'm glad you're feeling good!
--------------------------------------------------
You: exit
Bot: 👋 Goodbye!
